In [1]:
import os
import sys
import glob
import numpy as np

from parse import load_ps

import matplotlib.pyplot as plt

In [2]:
def split_num(s):
    head = s.rstrip('0123456789')
    tail = s[len(head):]
    return head, tail

In [3]:
def files_in_order(folderpath):
    npy_files = os.listdir(folderpath)

    no_extensions = [os.path.splitext(npy_file)[0] for npy_file in npy_files]

    splitted = [split_num(s) for s in no_extensions]

    splitted = np.array(splitted)

    indices = np.lexsort((splitted[:, 1].astype(int), splitted[:, 0]))

    npy_files = np.array(npy_files)
    return npy_files[indices]

In [5]:
files = files_in_order('poses_compressed\\shoulderpress')
print(files)

['shoulderpressbad1.npy' 'shoulderpressbad2.npy' 'shoulderpressbad3.npy'
 'shoulderpressbad4.npy' 'shoulderpressbad5.npy' 'shoulderpressbad6.npy'
 'shoulderpressbad7.npy' 'shoulderpressbad8.npy' 'shoulderpressbad9.npy'
 'shoulderpressbad10.npy' 'shoulderpressbad11.npy'
 'shoulderpressbad12.npy' 'shoulderpressbad13.npy'
 'shoulderpressbad14.npy' 'shoulderpressbad15.npy'
 'shoulderpressbad16.npy' 'shoulderpressbad17.npy'
 'shoulderpressgood1.npy' 'shoulderpressgood2.npy'
 'shoulderpressgood3.npy' 'shoulderpressgood4.npy'
 'shoulderpressgood5.npy' 'shoulderpressgood6.npy'
 'shoulderpressgood7.npy' 'shoulderpressgood8.npy'
 'shoulderpressgood9.npy' 'shoulderpressgood10.npy'
 'shoulderpressgood11.npy' 'shoulderpressgood12.npy'
 'shoulderpressgood13.npy' 'shoulderpressgood14.npy'
 'shoulderpressgood15.npy' 'shoulderpressgood16.npy'
 'shoulderpressgood17.npy' 'shoulderpressgood18.npy'
 'shoulderpressgood19.npy']


In [1]:
for filename in files:
    print("="*30)
    print("Starting:", filename)
    ps = load_ps("poses_compressed\\shoulderpress\\"+filename)
    poses = ps.poses
    
    right_present = [1 for pose in poses 
            if pose.rshoulder.exists and pose.relbow.exists and pose.rwrist.exists]
    left_present = [1 for pose in poses
            if pose.lshoulder.exists and pose.lelbow.exists and pose.lwrist.exists]
    right_count = sum(right_present)
    left_count = sum(left_present)
    side = 'right' if right_count > left_count else 'left'

    print('Exercise arm detected as: {}.'.format(side))
    
    if side == 'right':
        joints = [(pose.rshoulder, pose.relbow, pose.rwrist, pose.rhip, pose.neck) for pose in poses]
    else:
        joints = [(pose.lshoulder, pose.lelbow, pose.lwrist, pose.lhip, pose.neck) for pose in poses]

    # filter out data points where a part does not exist
    joints = [joint for joint in joints if all(part.exists for part in joint)]
    joints_ = np.array(joints)
    
    # Neck to hip
    back_vec = np.array([(joint[4].x - joint[3].x, joint[4].y - joint[3].y) for joint in joints])
    # Check range of motion of the back
    # Straining back
    back_vec_range = np.max(back_vec, axis=0) - np.min(back_vec, axis=0)
    print("Range of motion for back: %s" % back_vec_range)
    
    # Rolling shoulder too much
    elbow = joints_[:, 1]
    elbow_x = np.array([joint.x for joint in elbow])

    neck = joints_[:, 4]
    neck_x = np.array([joint.x for joint in neck])
        
    if side =='right':
        print("Minimum distance between elbow and neck: ", np.min(elbow_x - neck_x))
    else:
        print("Minimum distance between elbow and neck: ", np.min(neck_x - elbow_x))
    
    # Shoulder to elbow    
    upper_arm_vecs = np.array([(joint[0].x - joint[1].x, joint[0].y - joint[1].y) for joint in joints])
    # Elbow to wrist
    forearm_vecs = np.array([(joint[2].x - joint[1].x, joint[2].y - joint[1].y) for joint in joints])
    
    # normalize vectors
    upper_arm_vecs = upper_arm_vecs / np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)
    forearm_vecs = forearm_vecs / np.expand_dims(np.linalg.norm(forearm_vecs, axis=1), axis=1)
    
    # Check if raised all the way up
    upper_arm_forearm_angles = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_arm_vecs, forearm_vecs), axis=1), -1.0, 1.0)))
    print("Max upper arm and forearm angle: ", np.max(upper_arm_forearm_angles))
    


NameError: name 'files' is not defined